In [33]:
# Parsing animaldb ontology into n3 format
import rdflib

g = rdflib.Graph()
g.parse('animaldb.ttl', format='n3')

<Graph identifier=N9b3fc85786ed4c98983d04ff345f176a (<class 'rdflib.graph.Graph'>)>

In [34]:
#Checking if the folium map works correctly

import folium
m=folium.Map(location=[51.5,-0.1], zoom_start=5)
m

In [37]:
# Query all countries, lat, geo and total animal count
qres = g.query("""

PREFIX a: <http://example.com/animal/>
PREFIX spif: <http://spinrdf.org/spif#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX : <http://www.semanticweb.org/eduard/ontologies/2020/9/animal#>

select ?country ?lat ?long ?name (COUNT(?animal) as ?totalcount) where { 
    ?country a a:Country .
    ?country rdfs:label ?name .
    ?country geo:lat ?lat .
    ?country geo:long ?long .
    ?animal a:livesIn ?country .
} group by ?country ?lat ?long ?name
""")

# Query total species count per country
speciesquery = g.query ( """

PREFIX a: <http://example.com/animal/>
PREFIX spif: <http://spinrdf.org/spif#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

select ?country ?species (COUNT(?animal) as ?animalcount) where {
    ?country a a:Country .
    ?animal a:livesIn ?country .
    ?animal a:isSpecies ?speciesuri .
    ?speciesuri rdfs:label ?species
} GROUP BY ?country ?species

""")

# Iterate over every country, and add bubble marker information, as well as total species count for each country
for row1 in qres:
    
    tooltip = str(row1['name'] + ' has total animals: ' + row1['totalcount'] + ' of which: ')
    for row2 in speciesquery:
        if row1['country'] == row2['country']:
            tooltip += str('\n' + row2['animalcount'] + ' are ' + str(row2['species']) + ', ')
    folium.Circle(
        location = [row1['lat'],row1['long']],
        popup = tooltip,
        radius = 50000,
        color='crimson',
        fill=True,
        fill_color='crimson'
    ).add_to(m)

In [39]:
m